In [1]:
# coding: utf-8
"""
rakuten_corpus用
単語の分裂なし
全ての評価者による文での決定を用いて評価
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [2]:
# エッジリストの読み込み
list_edge = readcsv("./files/rakuten_corpus/rakuten_corpus_edgelist_full.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=6).membership
# 元のネットワークのpagerankを求める
dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["center_bet"] = g_sub.betweenness(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["center_eigen"] = g_sub.eigenvector_centrality(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
print "クラスタ数: ", len(list_dict_network_sub)

# トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
list_dict_prob = []
for i in range(len(list_dict_network_sub)):
    list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
    list_dict_prob.append({row[0]: row[1] for row in list_word_page})
    
# 凸２次計画問題を解いて、p(topic)を求める
list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)

クラスタ数:  6

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.8731e-03 -1.0035e+00  1e+00  8e-17  3e+00
 1: -2.8736e-03 -1.3473e-02  1e-02  1e-16  3e-02
 2: -2.9026e-03 -3.4281e-03  5e-04  1e-16  1e-03
 3: -2.9481e-03 -2.9876e-03  4e-05  8e-17  3e-05
 4: -2.9499e-03 -2.9510e-03  1e-06  1e-16  5e-07
 5: -2.9499e-03 -2.9499e-03  1e-08  8e-17  5e-09
 6: -2.9499e-03 -2.9499e-03  1e-10  1e-16  5e-11
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.07 	CPU Time Elapsed = 0.01
objFuncValue: -0.0029498728 (feasible, MaxResidual = 0)


In [19]:
num = 5
list_tmp = sorted(list_dict_prob[num].items(), key=lambda x: x[1], reverse=True)
for i in range(10):
    print list_tmp[i][0], list_tmp[i][1]

朝食 0.0433394285392
食事 0.0232707562762
美味しい 0.0135378638178
温泉 0.0131603111731
おいしい 0.0110730694262
多い 0.0108251541743
無料 0.0096936933632
人 0.00922968644611
料理 0.00902433142529
良い 0.00864968437555


### 定量評価部分
1. 形態素解析済みセンテンスを読み込み、確率を計算し、どのクラスに属するか計算
2. 予測クラスと実際クラスのリストを作成する

In [9]:
list_dict_words = dict_network_master["vertex"]

In [10]:
list_sep_words = readdump("./files/rakuten_corpus/annotation/all_sep_full.dump")
list_sep_words_rev = []
for row in list_sep_words:
    list_sep_words_rev.append([row[2], row[1]])

In [11]:
print len(list_sep_words)
print len(list_sep_words_rev)

2044
2044


In [12]:
# 確率が最大になるクラスを予想する
# 実質ラベルがintじゃない場合は、エラーとして、記録しない
list_predict_measure = []
list_words_rev = []
error_count = 0
for row in list_sep_words_rev:
    try:
        class_tmp = 0
        prob_tmp = 0
        predict_class = []
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row[0]:
                prob_tmp_tmp *= float(dict_prob[word])
            prob_tmp_tmp *= list_prob_topic[num]
            if prob_tmp_tmp > prob_tmp:
                class_tmp = num
                prob_tmp = prob_tmp_tmp
        list_predict_measure.append([class_tmp, row[1]])
    except:
        error_count += 1
print "エラーデータ数: ", error_count

エラーデータ数:  2


###計算結果の表示
* 6クラス

In [19]:
cal_f_measure(list_predict_measure)

Purity:  0.593093093093
Inverse Purity:  0.650650650651
F-value:  0.62054005716


###計算結果の表示
* 10クラス

In [12]:
cal_f_measure(list_predict_measure)

Purity:  0.596596596597
Inverse Purity:  0.65015015015
F-value:  0.622223186655


### 計算結果の表示
* 6クラス、full_version

In [13]:
cal_f_measure(list_predict_measure)

Purity:  0.571498530852
Inverse Purity:  0.624877571009
F-value:  0.596997237304
